In [3]:
import timm
print(timm.__version__)

import torch
import torchvision

import pandas as pd


0.9.5


## 軽量っぽいモデルをピックアップ
モデル名に ``nano``, ``small``, ``tiny`` を含むモデルを抽出。
さらに、入力画像サイズが 224x224 のものを抽出。

In [13]:
nano_models = timm.list_models('*nano*', pretrained=True)
small_models = timm.list_models('*small*', pretrained=True)
tiny_models = timm.list_models('*tiny*', pretrained=True)

lightweight_models = list(set(nano_models + small_models + tiny_models))
lightweight_models.sort()
print(lightweight_models)
print(len(lightweight_models))

['coat_lite_small.in1k', 'coat_lite_tiny.in1k', 'coat_small.in1k', 'coat_tiny.in1k', 'coatnet_nano_rw_224.sw_in1k', 'coatnet_rmlp_nano_rw_224.sw_in1k', 'coatnext_nano_rw_224.sw_in1k', 'convit_small.fb_in1k', 'convit_tiny.fb_in1k', 'convnext_nano.d1h_in1k', 'convnext_nano.in12k', 'convnext_nano.in12k_ft_in1k', 'convnext_nano_ols.d1h_in1k', 'convnext_small.fb_in1k', 'convnext_small.fb_in22k', 'convnext_small.fb_in22k_ft_in1k', 'convnext_small.fb_in22k_ft_in1k_384', 'convnext_small.in12k', 'convnext_small.in12k_ft_in1k', 'convnext_small.in12k_ft_in1k_384', 'convnext_tiny.fb_in1k', 'convnext_tiny.fb_in22k', 'convnext_tiny.fb_in22k_ft_in1k', 'convnext_tiny.fb_in22k_ft_in1k_384', 'convnext_tiny.in12k', 'convnext_tiny.in12k_ft_in1k', 'convnext_tiny.in12k_ft_in1k_384', 'convnext_tiny_hnf.a2h_in1k', 'convnextv2_nano.fcmae', 'convnextv2_nano.fcmae_ft_in1k', 'convnextv2_nano.fcmae_ft_in22k_in1k', 'convnextv2_nano.fcmae_ft_in22k_in1k_384', 'convnextv2_tiny.fcmae', 'convnextv2_tiny.fcmae_ft_in1k', 

In [14]:
lightweight_models_224x224 = []
for model_name in lightweight_models:
    model = timm.create_model(model_name)
    if model.default_cfg['input_size'] == (3, 224, 224):
        print(model_name)
        lightweight_models_224x224.append(model_name)


coat_lite_small.in1k
coat_lite_tiny.in1k
coat_small.in1k
coat_tiny.in1k
coatnet_nano_rw_224.sw_in1k
coatnet_rmlp_nano_rw_224.sw_in1k
coatnext_nano_rw_224.sw_in1k
convit_small.fb_in1k
convit_tiny.fb_in1k
convnext_nano.d1h_in1k
convnext_nano.in12k
convnext_nano.in12k_ft_in1k
convnext_nano_ols.d1h_in1k
convnext_small.fb_in1k
convnext_small.fb_in22k
convnext_small.fb_in22k_ft_in1k
convnext_small.in12k
convnext_small.in12k_ft_in1k
convnext_tiny.fb_in1k
convnext_tiny.fb_in22k
convnext_tiny.fb_in22k_ft_in1k
convnext_tiny.in12k
convnext_tiny.in12k_ft_in1k
convnext_tiny_hnf.a2h_in1k
convnextv2_nano.fcmae
convnextv2_nano.fcmae_ft_in1k
convnextv2_nano.fcmae_ft_in22k_in1k
convnextv2_tiny.fcmae
convnextv2_tiny.fcmae_ft_in1k
convnextv2_tiny.fcmae_ft_in22k_in1k
davit_small.msft_in1k
davit_tiny.msft_in1k
deit3_small_patch16_224.fb_in1k
deit3_small_patch16_224.fb_in22k_ft_in1k
deit_small_distilled_patch16_224.fb_in1k
deit_small_patch16_224.fb_in1k
deit_tiny_distilled_patch16_224.fb_in1k
deit_tiny_patch

## CPUでの推論速度を測定してみる  

In [15]:
!lscpu

アーキテクチャ:                        x86_64
  CPU 操作モード:                      32-bit, 64-bit
  Address sizes:                       39 bits physical, 48 bits virtual
  バイト順序:                          Little Endian
CPU:                                   8
  オンラインになっている CPU のリスト: 0-7
ベンダー ID:                           GenuineIntel
  モデル名:                            Intel(R) Core(TM) i7-6700 CPU @ 3.40GHz
    CPU ファミリー:                    6
    モデル:                            94
    コアあたりのスレッド数:            2
    ソケットあたりのコア数:            4
    ソケット数:                        1
    ステッピング:                      3
    CPU 最大 MHz:                      4000.0000
    CPU 最小 MHz:                      800.0000
    BogoMIPS:                          6799.81
    フラグ:                            fpu vme de pse tsc msr pae mce cx8 apic s
                                       ep mtrr pge mca cmov pat pse36 clflush dt
                                       s acpi mmx fxsr sse sse2 ss ht tm pbe sys
             

In [19]:
from tqdm.notebook import tqdm
import time

df = pd.DataFrame(columns=['nodel_name', 'num', 'time'])
for model_name in tqdm(lightweight_models_224x224):
    model = timm.create_model(model_name, pretrained=True)
    model.to('cpu')
    model.eval()
    x = torch.randn(1, 3, 224, 224)
    
    for i in range(100):
        start = time.perf_counter()
        result = model(x)
        end = time.perf_counter()
        tmp = pd.DataFrame(data=[[model_name, i, end-start]], columns=['nodel_name', 'num', 'time'])
        df = pd.cancat([df, tmp], ignore_index=True, axis=0)
    
    del model


  0%|          | 0/117 [00:00<?, ?it/s]

/tmp/ipykernel_7597/1959956034.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append([[model_name, i, end-start]], ignore_index=True)
/tmp/ipykernel_7597/1959956034.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append([[model_name, i, end-start]], ignore_index=True)
/tmp/ipykernel_7597/1959956034.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append([[model_name, i, end-start]], ignore_index=True)
/tmp/ipykernel_7597/1959956034.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append([[model_name, i, end-start]], ignore_index=True)
/tmp/ipykernel_7597/1959956034.py:15: FutureWarning: The frame.append me

/tmp/ipykernel_7597/1959956034.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append([[model_name, i, end-start]], ignore_index=True)
/tmp/ipykernel_7597/1959956034.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append([[model_name, i, end-start]], ignore_index=True)
/tmp/ipykernel_7597/1959956034.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append([[model_name, i, end-start]], ignore_index=True)
/tmp/ipykernel_7597/1959956034.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append([[model_name, i, end-start]], ignore_index=True)
/tmp/ipykernel_7597/1959956034.py:15: FutureWarning: The frame.append me

/tmp/ipykernel_7597/1959956034.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append([[model_name, i, end-start]], ignore_index=True)
/tmp/ipykernel_7597/1959956034.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append([[model_name, i, end-start]], ignore_index=True)
/tmp/ipykernel_7597/1959956034.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append([[model_name, i, end-start]], ignore_index=True)
/tmp/ipykernel_7597/1959956034.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df.append([[model_name, i, end-start]], ignore_index=True)
/tmp/ipykernel_7597/1959956034.py:15: FutureWarning: The frame.append me

KeyboardInterrupt: 

In [20]:
df

,nodel_name,num,time
